For interacting with the new template format while drafting proposals.

In [2]:
#%%
import os
import re
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import requests
import xlwings as xw

import functions
import hide

In [190]:

wb = xw.books.active
print(wb)
ws = wb.sheets.active
print(ws)

<Book [J12513 MODEC - BM-C-33 FPSO - NL WHISTLE B0.xlsx]>
<Sheet [J12513 MODEC - BM-C-33 FPSO - NL WHISTLE B0.xlsx]NAVIGATION LIGHTS>


In [ ]:
functions.hide_columns(ws)

In [ ]:
functions.format(ws)

In [ ]:
functions.fill_formula(ws)

In [5]:
functions.format(ws)

In [ ]:
functions.number_title(wb)

In [13]:
sheet = wb.sheets.active
last_row = sheet.range('G100000').end('up').row
sheet.page_setup.print_area = 'A1:H' + str(last_row)

In [35]:
# Set print area
skip_sheets = ['Config', 'Cover', 'Summary', 'Technical_Notes', 'T&C']
for sheet in wb.sheet_names:
    if sheet not in skip_sheets:
        ws = wb.sheets[sheet]
        last_row = ws.range('G100000').end('up').row
        ws.page_setup.print_area = 'A1:H' + str(last_row)


In [7]:
# Fill formula and format for the whole workbook.
skip_sheets = ['Config', 'Cover', 'Summary', 'Technical_Notes', 'T&C']
for sheet in wb.sheet_names:
    if sheet not in skip_sheets:
        functions.fill_formula(wb, sheet)
        functions.format(wb, sheet)
